In [128]:
import pandas as pd
from assign_unique_ids_functions import *
import inspect
import itertools

In [129]:
def intersect(a, b):
     return list(set(a) & set(b))
def setdiff(a,b):
    return list(set(a) - set(b))
def union(a,b):
    return list(set(a) | set(b))

In [130]:
#uhd = pd.read_csv("input/unit-history_demographics.csv.gz")
#amd = pd.read_csv("input/all-members_demographics.csv.gz")
asd = pd.read_csv("input/all-sworn_demographics.csv.gz")
asud = pd.read_csv("input/all-sworn-units_demographics.csv.gz")

In [131]:
TakeFirst4 = lambda x: x[:4]
BY_to_CA = lambda x: 2016 - x
def AddColumns(df, add_cols = ["F4FN", "F4LN", "Current.Age", "CA_BY"]):
    if "F4FN" in add_cols and "First.Name" in df.columns:
            df['F4FN'] = df['First.Name'].map(TakeFirst4)
    if "F4LN" in add_cols and 'Last.Name' in df.columns:
            df['F4LN'] = df['Last.Name'].map(TakeFirst4)
    if "Current.Age" in add_cols and "Current.Age" in df.columns:
        df['Current.Age.p1'] = df['Current.Age']
        df['Current.Age.m1'] = df['Current.Age']
    if "CA_BY" in add_cols and "Birth.Year" in df.columns:
        by_to_ca = lambda x: 2016 - x
        df['Current.Age.p1'] = df['Birth.Year'].map(BY_to_CA)
        df['Current.Age.m1'] = df['Birth.Year'].map(BY_to_CA) - 1  
    return df

In [132]:
asd_prep = asd.dropna(axis=1, how='all')
asud_prep = asud.dropna(axis=1, how='all')
asd_prep = AddColumns(asd_prep, add_cols = ['F4FN', 'F4LN'])
asud_prep = AddColumns(asud_prep, add_cols = ['F4FN', 'F4LN'])

In [155]:
asud_prep.shape

(31170, 11)

In [156]:
asd_prep.shape

(32009, 11)

In [135]:
def merge_datasets(df1, df2, add_cols, keep_cols):
    base_lists = [
        ['Current.Star', 'Star1', 'Star2', 'Star3', 'Star4', 'Star5','Star6', 'Star7', 'Star8', 'Star9', 'Star10'],
        ['First.Name', 'F4FN'], ['Last.Name', 'F4LN'], ['Appointed.Date'],
        ['Birth.Year', 'Current.Age', 'Current.Age.p1', 'Current.Age.m1', ''],
        ['Middle.Initial', ''], ['Gender', ''], ['Race', ''], ['Suffix.Name', '']
    ]
    
    df1 = df1.dropna(axis=1, how='all')
    df2 = df2.dropna(axis=1, how='all')

    df1 = AddColumns(df1, add_cols)
    df2 = AddColumns(df2, add_cols)
    
    cols = intersect(df1.columns, df2.columns)
    
    df1 = df1[[col for col in df1.columns
               if col in cols or col == keep_cols[0]]]
    df2 = df2[[col for col in df2.columns
               if col in cols or col == keep_cols[1]]]
    
    merge_list = []
    for col_list in base_lists:
        if intersect(col_list, cols):
            ml = intersect(col_list, cols)
            if '' in col_list:
                ml.append('')
            merge_list.append(sorted(ml, reverse=True))

    merge_list = list(itertools.product(*reversed(merge_list)))
    merge_list = [[i for i in ml if i != ''] for ml in merge_list]
    
    LoopMerge(df1, df2, on_list=merge_list, keep_columns = keep_cols)

['Last.Name',
 'Middle.Initial',
 'F4LN',
 'First.Name',
 'Appointed.Date',
 'Suffix.Name',
 'Gender',
 'Race',
 'F4FN',
 'Birth.Year']

In [136]:
asd_prep = asd_prep[[col for col in asd_prep.columns
                     if col in cols or '_ID' in col]]
asud_prep = asud_prep[[col for col in asud_prep.columns
                     if col in cols or '_ID' in col]]

In [137]:
cols

['Last.Name',
 'Middle.Initial',
 'F4LN',
 'First.Name',
 'Appointed.Date',
 'Suffix.Name',
 'Gender',
 'Race',
 'F4FN',
 'Birth.Year']

In [147]:

print(merge_list)

[['First.Name', 'F4FN'], ['Last.Name', 'F4LN'], ['Appointed.Date'], ['Birth.Year', ''], ['Middle.Initial', ''], ['Gender', ''], ['Race', ''], ['Suffix.Name', '']]


In [150]:
merge_list

[['Suffix.Name',
  'Race',
  'Gender',
  'Middle.Initial',
  'Birth.Year',
  'Appointed.Date',
  'Last.Name',
  'First.Name'],
 ['Suffix.Name',
  'Race',
  'Gender',
  'Middle.Initial',
  'Birth.Year',
  'Appointed.Date',
  'Last.Name',
  'F4FN'],
 ['Suffix.Name',
  'Race',
  'Gender',
  'Middle.Initial',
  'Birth.Year',
  'Appointed.Date',
  'F4LN',
  'First.Name'],
 ['Suffix.Name',
  'Race',
  'Gender',
  'Middle.Initial',
  'Birth.Year',
  'Appointed.Date',
  'F4LN',
  'F4FN'],
 ['Suffix.Name',
  'Race',
  'Gender',
  'Middle.Initial',
  'Appointed.Date',
  'Last.Name',
  'First.Name'],
 ['Suffix.Name',
  'Race',
  'Gender',
  'Middle.Initial',
  'Appointed.Date',
  'Last.Name',
  'F4FN'],
 ['Suffix.Name',
  'Race',
  'Gender',
  'Middle.Initial',
  'Appointed.Date',
  'F4LN',
  'First.Name'],
 ['Suffix.Name',
  'Race',
  'Gender',
  'Middle.Initial',
  'Appointed.Date',
  'F4LN',
  'F4FN'],
 ['Suffix.Name',
  'Race',
  'Gender',
  'Birth.Year',
  'Appointed.Date',
  'Last.Name',
  

In [153]:
def LoopMerge(df1, df2, on_list, keep_columns = ['ID1', 'ID2'], return_unmatched = True):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    # Generate on list?
    for mc in on_list:
        df1t = remove_duplicates(df1[keep_columns[:1] + mc], mc)
        df2t = remove_duplicates(df2[keep_columns[1:] + mc], mc)
        dfmt = df1t.merge(df2t, on=mc, how='inner')
        if dfmt.shape[0] > 0:
            print('******')
            print(mc)
            print(dfmt.shape[0])
            print('******')
            dfmt['Match'] = '-'.join(mc)
            dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
            df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
            df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
    if return_unmatched:
        return (dfm.reset_index(drop=True), df1, df2)
    else:
        return dfm.reset_index(drop=True)

In [154]:
ml = LoopMerge(asud_prep, asd_prep, on_list=merge_list, keep_columns = ['all_sworn_units_ID', 'all_sworn_ID'])

******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'Last.Name', 'First.Name']
31170
******


In [ ]:
def StarMerge(df1, df2, on_list, keep_columns, return_unmerged = True):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    df2 = df2.loc[(df2["Star"].notnull()) & (df2["Star"] > 0)]
    stars = ["Star" + str(i) for i in range(1,11)]
    for mc_cols in on_list:
        for star in stars:
            mc1 = mc_cols + [star]
            mc2 = mc_cols + ["Star"]
            df1t = RemoveDuplicates(df1.loc[(df1[star].notnull()) & (df1[star] > 0), keep_columns[:1] + mc1], mc1)
            df2t = RemoveDuplicates(df2[keep_columns[1:] + mc2], mc2)
            dfmt = df1t.merge(df2t, left_on = mc1, right_on = mc2, how='inner')
            if dfmt.shape[0] > 0:
                print('******')
                print(mc1)
                print(dfmt.shape[0])
                print('******')
                dfmt['Match'] = '-'.join(mc + [star + "/" + "Star"])
                dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
                df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
                df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
    if return_unmerged:
        return (dfm.reset_index(drop=True), df1, df2)
    else:
        return dfm.reset_index(drop=True)